<a href="https://colab.research.google.com/github/ramsundar619/Text_Classifier_BERT/blob/main/Text_Classifier_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [ ]:
from google.colab import drive                                     #mounting the drive to acces the data stored in google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Acies/Analysis_Chart_Configs.csv')

In [ ]:
df.head()

,Chart name,Chart config,Description
0,Vertical Bar Chart,"{""chart_name"":""Vertical Bar"",""table"":""Trade_Du...",Plot Isin against Quantity
1,Vertical Bar Chart,"{""chart_name"":""Vertical Bar"",""table"":""Trade_Du...",Isin wise Quantity plot
2,Vertical Bar Chart,"{""chart_name"":""Vertical Bar"",""table"":""Trade"",""...",Isin wise Quantity plot using Trade table
3,Vertical Bar Chart,"{""chart_name"":""Vertical Bar"",""table"":""Trade_Du...",Isin wise Quantity plot using Trade_Dummy table
4,Vertical Bar Chart,"{""chart_name"":""Vertical Bar"",""table"":""Trade_Du...",Vertical Bar with Isin in x_axis and Quantity ...


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'Vertical Bar Chart':0,
          'Pie Chart':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['Chart name']]
        self.texts = [tokenizer(Description, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for Description in df['Description']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
df_train, df_val = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df))])

In [ ]:
print(len(df_train),len(df_val))

26 7


In [ ]:
EPOCHS = 20
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 13/13 [00:05<00:00,  2.37it/s]


Epochs: 1 | Train Loss:  0.769 | Train Accuracy:  0.269 | Val Loss:  0.976 | Val Accuracy:  0.286


100%|██████████| 13/13 [00:05<00:00,  2.36it/s]


Epochs: 2 | Train Loss:  0.745 | Train Accuracy:  0.385 | Val Loss:  0.886 | Val Accuracy:  0.429


100%|██████████| 13/13 [00:05<00:00,  2.37it/s]


...

Epochs: 18 | Train Loss:  0.543 | Train Accuracy:  0.769 | Val Loss:  0.534 | Val Accuracy:  0.857


100%|██████████| 13/13 [00:05<00:00,  2.37it/s]


Epochs: 19 | Train Loss:  0.554 | Train Accuracy:  0.692 | Val Loss:  0.749 | Val Accuracy:  0.571


100%|██████████| 13/13 [00:05<00:00,  2.42it/s]


Epochs: 20 | Train Loss:  0.525 | Train Accuracy:  0.731 | Val Loss:  0.817 | Val Accuracy:  0.429


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'Pie Chart for Quanty by Isin'
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt")

In [ ]:
bert_input

{'input_ids': tensor([[  101, 26424,  6900,  1111,   154, 27280,  1183,  1118,  2181,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
  model = model.cuda()
output = model(bert_input.input_ids.to(device),bert_input.attention_mask.to(device))

In [ ]:
print(output.argmax(dim=1))

tensor([0], device='cuda:0')
